Important Links:
<br>[Med Cabinet 5](https://github.com/Build-Week-Med-Cabinet-5)
<br>[Med Cabinet 4](https://github.com/Build-Week-Med4)
<br>[Trello: MC4](https://trello.com/b/VDYFGdL6/bw-med4-team)

What I need to do
* [x] import the csv
* [x] clean up the data
* get a list of unique
  * flavors
  * effects
* split the data 
* pretrain or not 
* start playing with models
  * NLP? 
  * RNN? 
  * K-means clustering?
  * PCA?

# Cleaning Cannabis CSV

In [1]:
import pandas as pd


In [2]:
# reading data
URL = 'https://raw.githubusercontent.com/Build-Week-Med4/DS/master/cannabis.csv'
strains_df = pd.read_csv(URL, na_values="None")
strains_df.columns= strains_df.columns.str.lower()

In [3]:
# shape and first look at df
print(strains_df.shape)
strains_df.head()

(2351, 6)


,strain,type,rating,effects,flavor,description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [4]:
# where are all the NaNs
strains_df.loc[strains_df.isnull().any(axis=1)]

,strain,type,rating,effects,flavor,description
5,3-Bears-Og,indica,0.0,NaN,NaN,3 Bears OG by Mephisto Genetics is an autoflow...
16,831-Og,indica,5.0,NaN,NaN,"831 OG, aka Santa Cruz OG, is an indica-domina..."
32,Acid-Dough,sativa,5.0,"Talkative,Giggly,Happy,Hungry,Relaxed",NaN,Acid Dough by Ripper Seeds is a sativa-dominan...
54,Afrikaner,sativa,0.0,NaN,NaN,Afrikaner by Cannabaal and master grower Waldo...
60,Als-Dream,hybrid,3.0,"Talkative,Energetic",NaN,Al’s Dream is rapidly becoming a Colorado cann...
...,...,...,...,...,...,...
2298,White-Ryder,hybrid,0.0,NaN,NaN,White Ryder is Riot Seeds’ attempt to create a...
2309,White-Walrus,hybrid,2.5,"Talkative,Uplifted,Creative,Energetic,Euphoric",NaN,"White Walrus is a phenotype of Chernobyl, grow..."
2321,Wonderwoman-Og,indica,4.0,"Relaxed,Tingly,Euphoric,Aroused,Happy","Flowery,Menthol,Citrus",NaN
2327,X-File,hybrid,4.5,"Tingly,Euphoric,Relaxed",NaN,Not to be confused with the similar-sounding X...


In [5]:
# drop the NaNs
strains_df = strains_df.dropna().reset_index()
print(strains_df.shape)
strains_df.head()

(2163, 7)


,index,strain,type,rating,effects,flavor,description
0,0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


# Combining Features
`effects` + `flavors` + `description`

In [6]:
# make combined feature
strains_df['combined'] = strains_df['effects'] + \
                          strains_df['flavor'] + \
                          strains_df['description']

In [7]:
strains_df['combined'][6]

'Relaxed,Happy,Euphoric,Uplifted,GigglyCitrus,Pungent,EarthyThe indica-dominant 303 OG is a Colorado strain bred by crossing Pre-98 Bubba Kush and Chemdawg. Its parent strains pass on a complex flavor profile of coffee and spice with diesel undertones. Its buzz can be described as social and creative, a dreamy experience perfect for letting off steam at the end of the work day. The happy euphoria induced by 303 OG may help patients treating depression, anxiety, and stress disorders.'

In [8]:
# drop extra index column
strains_df = strains_df.drop(['index'], axis=1)
strains_df.head()

,strain,type,rating,effects,flavor,description,combined
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,"Creative,Energetic,Tingly,Euphoric,RelaxedEart..."
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,"Relaxed,Aroused,Creative,Happy,EnergeticFlower..."
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,"Uplifted,Happy,Relaxed,Energetic,CreativeSpicy..."
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,"Tingly,Creative,Hungry,Relaxed,UpliftedApricot..."
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...","Happy,Relaxed,Euphoric,Uplifted,TalkativeCitru..."


# Tokenize the Combined Column

In [9]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [10]:
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_lg")

# The Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Make the tokens for description
combined_tokens = []
for txt in tokenizer.pipe(strains_df['combined'], batch_size=500):
    txt_tokens = [token.text for token in txt]
    combined_tokens.append(txt_tokens)
strains_df['combined_tokens'] = combined_tokens
print(strains_df['combined_tokens'].head())

0    [Creative,Energetic,Tingly,Euphoric,RelaxedEar...
1    [Relaxed,Aroused,Creative,Happy,EnergeticFlowe...
2    [Uplifted,Happy,Relaxed,Energetic,CreativeSpic...
3    [Tingly,Creative,Hungry,Relaxed,UpliftedAprico...
4    [Happy,Relaxed,Euphoric,Uplifted,TalkativeCitr...
Name: combined_tokens, dtype: object


In [11]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) \
            and (token.is_punct != True)]

In [13]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                       ngram_range = (1,2),
                       max_features = 2000)

# Create a vocabulary and tf-idf score per document
dtm = tfidf.fit_transform(strains_df['combined'])
                         

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)


# View Feature Matrix as DataFrame
print(dtm.shape)
dtm.head()

(2163, 2000)


,10,10 weeks,11,11 weeks,12,13,14,15,18,1980s,1st,1st hawaiian,1st place,20,20 indica,20 thc,2012,2014,2014 cannabis,2015,2016,22,23,24,25,2nd,2nd place,30,30 indica,3rd,3rd place,40,40 indica,40 sativa,45,47,50,50 50,50 hybrid,55,...,walker,want,warm,washington,washington state,way,way cross,week,week flowering,weeks,weight,weighted,west,west coast,whimsical,white,white alien,white og,white widow,wide,widow,winner,winning,won,won 1st,wonder,woody,woody spicy,work,works,world,worth,wowie,wrapped,wreck,years,yield,yielding,yields,zesty
0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.286626,0.15769,0.162149,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.101565,0.132918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.291018,0.0,0.0,0.371532,0.0,0.369437,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.138621,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.35166,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.12036,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.133172,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.131099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
ideal = ["""
Creative,Uplifted,Tingly,Euphoric,Relaxed, Giggly
"""]

In [15]:
# Query the ideal descprition
new = tfidf.transform(ideal)
new

<1x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [16]:
nn.kneighbors(new.todense())

(array([[1.19045604, 1.26028088, 1.26830873, 1.27166822]]),
 array([[1881, 1586,  373,  317]]))

In [19]:
# most ideal description,strain and flavor
strains_df['description'][1681]

'Saturn OG is a hybrid strain with mysterious beginnings, but its earthy citrus and diesel flavors confirms its close relationship to OG Kush. Its undocumented origins result in this strain being labeled as indica, sativa, and everything in between, but its effects are most commonly described as hybrid-like in its balanced calm and moderate cerebral effects. Saturn OG’s forest green buds are lit by a constellation of crystal trichomes, and this OG Kush relative is often lumped into a “planetary strain” series that includes Earth OG and Jupiter OG. The psychoactive onset of Saturn OG begins with an intense burst of euphoria that fades to smooth relaxation perfect for relieving stress and muscle tension.\xa0'

# Pickle Time

In [20]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl2'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances

In [21]:
# Loading the saved model
model_pkl2 = open(pickle_filename, 'rb')
NN_model2 = pickle.load(model_pkl2)
print ("Loaded model :: ", NN_model2)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [22]:
# Dump the trained classifier (tfidf)  with Pickle
pickle_filename_1 = 'tfidf.pkl2'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [23]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model2 = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model2)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [24]:
ideal2 = ["happy,Sleepy,Apricot,Citrus,Grapefruit"]

In [25]:
new2 = tfidf_model2.transform(ideal2)
new2

<1x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [26]:
NN_model2.kneighbors(new2.todense())

(array([[1.09504845, 1.14089606, 1.15844355, 1.18178543]]),
 array([[1911,  398,  863, 1497]]))

In [31]:
import json
def recommend(user_input):
    temp_df = NN_model2.kneighbors(tfidf_model2.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = strains_df.loc[temp_df[0][i]]['strain']
        info_effects = strains_df.loc[temp_df[0][i]]['effects']
        info_flavor = strains_df.loc[temp_df[0][i]]['flavor']
        info_description = strains_df.loc[temp_df[0][i]]['description']
        info_rating = strains_df.loc[temp_df[0][i]]['rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info)  #for engineeers, the return does not work in jupyter lab.  Should work in vsCode.
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dump(info_description)
        #return json.dumps(info_rating)

In [32]:
recommend('hybrid-like in its balanced calm and moderate cerebral effects')

"Saturn-Og"
"Happy,Relaxed,Euphoric,Uplifted,Sleepy"
"Earthy,Sweet,Tree,Fruit"
"Saturn OG is a hybrid strain with mysterious beginnings, but its earthy citrus and diesel flavors confirms its close relationship to OG Kush. Its undocumented origins result in this strain being labeled as indica, sativa, and everything in between, but its effects are most commonly described as hybrid-like in its balanced calm and moderate cerebral effects. Saturn OG\u2019s forest green buds are lit by a constellation of crystal trichomes, and this OG Kush relative is often lumped into a \u201cplanetary strain\u201d series that includes Earth OG and Jupiter OG. The psychoactive onset of Saturn OG begins with an intense burst of euphoria that fades to smooth relaxation perfect for relieving stress and muscle tension.\u00a0"
4.5
"Moonwalker-Kush"
"Relaxed,Euphoric,Happy,Focused,Giggly"
"Earthy,Pine,Minty"
"Moonwalker Kush is a balanced indica-dominant hybrid that is the genetic cross of Triple OG and Tahoe Al